In [42]:
from datasets import Dataset, load_dataset
import pandas as pd
from PIL import Image
import numpy as np
from io import BytesIO
import random

In [2]:
dataset = load_dataset("A2H0H0R1/plant-disease-new", split="train")

In [40]:
len(dataset)

162916

In [45]:
random_indices = random.sample(range(0, 162917), 200)

In [46]:
len(random_indices)

200

In [47]:
data = []

for i in random_indices:
    img = np.array(dataset[i]['image'])
    label = dataset[i]['label']
    data.append({'image': img, 'label': label})

In [48]:
df = pd.DataFrame(data)

In [49]:
df.head()

,image,label
0,"[[[113, 113, 113], [99, 99, 99], [101, 101, 10...",29
1,"[[[119, 119, 119], [123, 123, 123], [133, 133,...",35
2,"[[[74, 76, 73], [75, 77, 74], [75, 77, 74], [7...",15
3,"[[[169, 201, 90], [167, 201, 91], [156, 189, 8...",10
4,"[[[68, 71, 76], [77, 80, 85], [85, 88, 93], [8...",35


In [50]:
df.shape

(200, 2)

In [51]:
len(df['label'].unique())

35

In [63]:
max(df['label'])

37

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [65]:
images = np.stack(df['image'].values)
labels = df['label'].values

In [66]:
images = images / 255.0
labels = to_categorical(labels, num_classes=38)

In [69]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(38, activation='softmax')
])

In [70]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [73]:
len(images), len(labels)

(200, 200)

In [72]:
history = model.fit(images, labels, epochs=10, batch_size=32)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 537ms/step - accuracy: 0.1184 - loss: 3.4713
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 520ms/step - accuracy: 0.1427 - loss: 3.2905
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 515ms/step - accuracy: 0.1899 - loss: 3.0433
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 512ms/step - accuracy: 0.2951 - loss: 2.5844
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 544ms/step - accuracy: 0.3910 - loss: 2.1338
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 583ms/step - accuracy: 0.5556 - loss: 1.7037
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 591ms/step - accuracy: 0.6826 - loss: 1.1855
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 557ms/step - accuracy: 0.7326 - loss: 0.9279
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 523ms/step - accuracy: 0.8030 - loss: 0.6314
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 521ms/step - accuracy: 0.8236 - loss: 0.5478


In [74]:
model.save('../cnn-models/model.keras')